In [1]:
import requests
import nba_api
from nba_api.stats.endpoints import (playbyplayv2,playbyplay,leaguehustlestatsplayer,
                                     leaguedashplayerbiostats,commonplayerinfo,leaguestandings)
import pandas as pd
import numpy as np
import time

In [2]:
# this method takes 2 minutes to append all dataframes (down from 12 minutes)
print(time.strftime("%H:%M:%S",time.localtime()))
seasons=list(range(1997,2022))
season_slug=[str(x)+'-'+str(x+1)[-2:] for x in seasons]
pbp=pd.read_csv(filepath_or_buffer="PBP Data/1996-97_pbp.csv")
for seas in season_slug:
    pbp=pbp.append(pd.read_csv(filepath_or_buffer="PBP Data/"+seas+"_pbp.csv"))
pbp=pbp.drop(columns=['VIDEO_AVAILABLE_FLAG','PLAYER3_TEAM_NICKNAME','PLAYER3_TEAM_CITY','PLAYER3_TEAM_ID',
                      'PLAYER2_TEAM_NICKNAME','PLAYER2_TEAM_CITY','PLAYER2_TEAM_ID','PLAYER1_TEAM_NICKNAME',
                      'PLAYER1_TEAM_CITY','PLAYER1_TEAM_ID','PERSON1TYPE','PERSON2TYPE','PERSON3TYPE','Unnamed: 0'],errors='ignore')
print(time.strftime("%H:%M:%S",time.localtime()))

10:11:56
10:13:40


In [3]:
# deleted because taking up too much space
# print(time.strftime("%H:%M:%S",time.localtime()))
# pbp=pd.read_csv(filepath_or_buffer="PBP Data/1997-2020_pbp.csv")
# print(time.strftime("%H:%M:%S",time.localtime()))

In [4]:
pbp['season']=pbp['GAME_ID'].astype(str).str[1:3].astype(int).add(1)

In [5]:
pbp

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,season
0,29600001,1,12,0,1,11:15 PM,12:00,NaN,Start of 1st Period (11:15 PM EST),NaN,...,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,97
1,29600001,2,10,0,1,11:16 PM,12:00,Jump Ball Ellison vs. Longley: Tip to Harper,NaN,NaN,...,442,Pervis Ellison,BOS,26,Luc Longley,CHI,166,Ron Harper,CHI,97
2,29600001,4,1,5,1,11:16 PM,11:39,NaN,NaN,Rodman Layup (2 PTS) (Longley 1 AST),...,23,Dennis Rodman,CHI,26,Luc Longley,CHI,0,NaN,BOS,97
3,29600001,5,6,2,1,11:16 PM,11:39,Ellison S.FOUL (P1.T1),NaN,NaN,...,442,Pervis Ellison,BOS,0,NaN,NaN,0,NaN,NaN,97
4,29600001,6,3,10,1,11:17 PM,11:39,NaN,NaN,Rodman Free Throw 1 of 1 (3 PTS),...,23,Dennis Rodman,CHI,0,NaN,NaN,0,NaN,NaN,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570251,22101230,670,2,1,4,12:03 AM,0:21,NaN,NaN,MISS Alexander-Walker 3PT Jump Shot,...,1629638,Nickeil Alexander-Walker,UTA,0,NaN,NaN,0,NaN,NaN,22
570252,22101230,671,4,0,4,12:03 AM,0:18,Perry REBOUND (Off:2 Def:6),NaN,NaN,...,1629617,Reggie Perry,POR,0,NaN,NaN,0,NaN,NaN,22
570253,22101230,672,2,1,4,12:03 AM,0:11,MISS Blevins 3PT Jump Shot,NaN,NaN,...,1629833,Keljin Blevins,POR,0,NaN,NaN,0,NaN,NaN,22
570254,22101230,673,4,0,4,12:03 AM,0:07,NaN,NaN,Sneed REBOUND (Off:1 Def:1),...,1630270,Xavier Sneed,UTA,0,NaN,NaN,0,NaN,NaN,22


In [6]:
alley_oops=pbp[pbp['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Alley',na=False)]

made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)].\
drop(columns=['PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_ABBREVIATION'])

alley_oops_w_2_players=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
alley_oops_w_2_players['p1']=alley_oops_w_2_players[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
alley_oops_w_2_players['p2']=alley_oops_w_2_players[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
alley_oops_w_2_players

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,SCOREMARGIN,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,season,p1,p2
769,20000520,367,1,52,4,3:51 PM,9:40,Mason Alley Oop Dunk (12 PTS) (Barry 4 AST),NaN,NaN,...,10,2046,Desmond Mason,SEA,699,Brent Barry,SEA,1,Brent Barry,Desmond Mason
1071,20000629,219,1,52,3,7:56 AM,10:30,NaN,NaN,Bryant Alley Oop Dunk (13 PTS) (Harper 2 AST),...,14,977,Kobe Bryant,LAL,166,Ron Harper,LAL,1,Kobe Bryant,Ron Harper
1272,20000629,431,1,43,4,8:54 AM,0:38,NaN,NaN,Bryant Alley Oop Layup (33 PTS) (Lue 4 AST),...,8,977,Kobe Bryant,LAL,1731,Tyronn Lue,LAL,1,Kobe Bryant,Tyronn Lue
1373,20000441,79,1,52,1,2:26 PM,3:36,Wallace Alley Oop Dunk (2 PTS) (Wells 1 AST),NaN,NaN,...,5,739,Rasheed Wallace,POR,1719,Bonzi Wells,POR,1,Bonzi Wells,Rasheed Wallace
1558,20000441,274,1,52,3,3:32 PM,5:36,Wallace Alley Oop Dunk (6 PTS) (Pippen 1 AST),NaN,NaN,...,10,739,Rasheed Wallace,POR,937,Scottie Pippen,POR,1,Rasheed Wallace,Scottie Pippen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569285,22101228,604,1,52,4,9:14 PM,4:01,Reed 1' Alley Oop Dunk (21 PTS) (Milton 4 AST),NaN,NaN,...,14,1630194,Paul Reed,PHI,1629003,Shake Milton,PHI,22,Paul Reed,Shake Milton
569566,22101229,359,1,52,3,10:56 PM,8:59,NaN,NaN,Jones Alley Oop Dunk (7 PTS) (Mitchell 10 AST),...,-13,1627745,Damian Jones,SAC,1630558,Davion Mitchell,SAC,22,Damian Jones,Davion Mitchell
569592,22101229,398,1,43,3,11:01 PM,6:17,Shamet 3' Alley Oop Layup (14 PTS) (Johnson 2 ...,NaN,NaN,...,-10,1629013,Landry Shamet,PHX,1629661,Cameron Johnson,PHX,22,Cameron Johnson,Landry Shamet
570132,22101230,496,1,52,3,11:32 PM,2:18,NaN,NaN,Gobert Alley Oop Dunk (14 PTS) (Gay 2 AST),...,-21,203497,Rudy Gobert,UTA,200752,Rudy Gay,UTA,22,Rudy Gay,Rudy Gobert


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [7]:
alley_oops_w_2_players.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(n=10,columns="count",keep="all")

,p1,p2,count
2783,Clint Capela,James Harden,360
2615,Chris Paul,DeAndre Jordan,258
1520,Blake Griffin,DeAndre Jordan,199
2649,Chris Paul,Tyson Chandler,197
2794,Clint Capela,Trae Young,161
604,Andre Drummond,Reggie Jackson,158
4477,Dwight Howard,Jameer Nelson,157
996,Anthony Davis,Jrue Holiday,139
4550,Dwight Powell,Luka Doncic,138
6740,John Collins,Trae Young,138


In [8]:
alley_oops_w_2_players.groupby(['season','p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(n=10,columns="count",keep="all")

,season,p1,p2,count
2971,8,Chris Paul,Tyson Chandler,107
9331,19,Clint Capela,James Harden,100
8560,18,Clint Capela,James Harden,98
11686,22,Clint Capela,Trae Young,92
7901,17,Clint Capela,James Harden,86
10852,21,Clint Capela,Trae Young,69
7263,16,Chris Paul,DeAndre Jordan,68
3437,9,Chris Paul,Tyson Chandler,65
11856,22,Dwight Powell,Luka Doncic,64
8461,18,Anthony Davis,Rajon Rondo,60


In [9]:
misses=pbp[pbp.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
blocked_dunks

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,season
12720,29600028,152,2,7,2,10:48 PM,8:04,Rogers BLOCK (1 BLK),NaN,MISS Fuller Dunk,...,957,Todd Fuller,GSW,0,NaN,NaN,964,Roy Rogers,VAN,97
17367,29600038,323,2,7,3,13:12 PM,2:17,MISS Owes Dunk,NaN,Dudley BLOCK (1 BLK),...,1077,Ray Owes,GSW,0,NaN,NaN,201,Chris Dudley,POR,97
19601,29600043,248,2,7,3,14:30 PM,9:20,Mourning BLOCK (3 BLK),NaN,MISS Jordan Dunk,...,893,Michael Jordan,CHI,0,NaN,NaN,297,Alonzo Mourning,MIA,97
23111,29600051,5,2,7,1,11:43 PM,11:06,Polynice BLOCK (1 BLK),NaN,MISS Smith Dunk,...,120,Steven Smith,ATL,0,NaN,NaN,178,Olden Polynice,SAC,97
33412,29600072,419,2,8,4,12:40 PM,4:09,MISS Jones 2' Slam Dunk,NaN,Mutombo BLOCK (3 BLK),...,224,Eddie Jones,LAL,0,NaN,NaN,87,Dikembe Mutombo,ATL,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564570,22101218,347,2,9,2,4:34 PM,1:01,MISS Mobley 1' Driving Dunk,NaN,Antetokounmpo BLOCK (1 BLK),...,1630596,Evan Mobley,CLE,0,NaN,NaN,203648,Thanasis Antetokounmpo,MIL,22
564913,22101219,157,2,9,2,10:09 PM,11:45,MISS Chriss 2' Driving Dunk,NaN,Collins BLOCK (1 BLK),...,1627737,Marquese Chriss,DAL,0,NaN,NaN,1628380,Zach Collins,SAS,22
567111,22101223,682,2,9,4,9:24 PM,0:41,Pons BLOCK (3 BLK),NaN,MISS Thomas 2' Driving Dunk,...,1630271,Brodric Thomas,BOS,0,NaN,NaN,1630582,Yves Pons,MEM,22
569569,22101229,364,2,108,3,10:57 PM,8:30,McGee BLOCK (1 BLK),NaN,MISS Lyles 2' Cutting Dunk Shot,...,1626168,Trey Lyles,SAC,0,NaN,NaN,201580,JaVale McGee,PHX,22


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blockee
blockee=blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'times_blocked','PLAYER1_NAME':'player'})
blockee.nlargest(n=10,columns="times_blocked",keep="all")

,player,times_blocked
1015,Rudy Gobert,61
1151,Tristan Thompson,49
488,Ivica Zubac,45
287,DeAndre Jordan,42
314,Derrick Favors,42
926,Pau Gasol,42
1014,Rudy Gay,42
357,Dwight Howard,41
1123,Tim Duncan,40
857,Montrezl Harrell,38


In [11]:
blocked_dunks.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'times_blocked','PLAYER1_NAME':'player'}).nlargest(n=10,columns="times_blocked",keep="all")

,season,player,times_blocked
2931,19,Rudy Gobert,15
1886,13,Tristan Thompson,13
1019,8,Emeka Okafor,12
3257,21,Ivica Zubac,12
2762,19,Andre Drummond,11
2830,19,Ivica Zubac,11
3551,22,Kenyon Martin Jr.,11
3607,22,Precious Achiuwa,11
316,3,Antonio Davis,10
1663,12,Kris Humphries,10


# The No Fly Zone Award*

most blocked dunks as the blocking player

In [12]:
#as the blocker
blocker=blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'times_blocker','PLAYER3_NAME':'player'})
blocker.nlargest(n=10,columns="times_blocker",keep="all")

,player,times_blocker
31,Alonzo Mourning,126
905,Serge Ibaka,123
285,Dikembe Mutombo,110
966,Tim Duncan,91
961,Theo Ratliff,90
518,Joel Przybilla,88
499,Jermaine O'Neal,87
329,Emeka Okafor,81
39,Andre Drummond,76
117,Brook Lopez,71


In [13]:
blocked_dunks.groupby(['season','PLAYER3_NAME']).size().reset_index().rename(columns={0:'times_blocker','PLAYER3_NAME':'player'}).nlargest(n=10,columns="times_blocker",keep="all")

,season,player,times_blocker
119,2,Alonzo Mourning,27
542,6,Alonzo Mourning,24
3290,100,Dikembe Mutombo,23
3271,100,Alonzo Mourning,22
485,5,Joel Przybilla,21
1877,16,Serge Ibaka,21
2226,19,Andre Drummond,21
2281,19,JaVale McGee,20
2744,21,Nerlens Noel,20
425,4,Theo Ratliff,18


In [14]:
# dunk_blocks=blocker.merge(blockee,on='player',how='outer').fillna(0)
# dunk_blocks['net_blocked_dunks']=dunk_blocks[['times_blocker']].subtract(dunk_blocks.times_blocked,axis=0)
# dunk_blocks['blocked_dunks_involved']=dunk_blocks[['times_blocker']].add(dunk_blocks.times_blocked,axis=0)
# dunk_blocks.sort_values('net_blocked_dunks',ascending=False).tail(10)
# dunk_blocks.sort_values('net_blocked_dunks',ascending=False).head(10)
# dunk_blocks.query('times_blocker == 0').sort_values('net_blocked_dunks',ascending=False).tail(10)

# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8))*

most charges committed (credit to Kdog122025 for the idea) (charges were first recorded starting in 2011)

In [15]:
charges=pbp[pbp['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
363,Giannis Antetokounmpo,210
887,Russell Westbrook,208
245,DeMarcus Cousins,186
453,James Harden,185
661,LeBron James,134
574,Julius Randle,123
823,Paul George,106
1024,Tyreke Evans,106
824,Paul Millsap,105
530,John Wall,102


In [16]:
charges.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER1_NAME,count
2816,19,Giannis Antetokounmpo,41
286,11,Russell Westbrook,39
3161,20,Giannis Antetokounmpo,34
2406,18,DeMarcus Cousins,30
2513,18,Julius Randle,30
89,11,DeMarcus Cousins,28
2885,19,Julius Randle,28
3497,21,Giannis Antetokounmpo,27
2450,18,Giannis Antetokounmpo,26
834,13,James Harden,25


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [17]:
goaltends=pbp[pbp['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
414,Dwight Howard,527
1207,Samuel Dalembert,351
576,JaVale McGee,279
52,Andre Drummond,214
697,Joel Przybilla,171
333,DeAndre Jordan,167
1226,Serge Ibaka,166
810,Kevin Garnett,165
1234,Shaquille O'Neal,149
802,Kenyon Martin,139


In [18]:
goaltends.groupby(['season','PLAYER1_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER1_NAME,count
1489,8,Dwight Howard,58
2075,11,Dwight Howard,57
2100,11,JaVale McGee,56
1686,9,Dwight Howard,53
1394,7,Samuel Dalembert,48
763,4,Samuel Dalembert,44
1876,10,Dwight Howard,44
1593,8,Samuel Dalembert,43
2493,13,Dwight Howard,43
899,5,Joel Przybilla,41


# The Most 3-Pt Shooting Fouls Committed Award*

most 3-point shooting fouls committed (credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea)

In [19]:
third_free_throws=pbp[pbp['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp[pbp['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp[pbp['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
712,Jeff Teague,59
893,Kobe Bryant,58
616,JR Smith,57
1175,Paul George,57
207,Carmelo Anthony,56
344,Darren Collison,55
809,Josh Richardson,55
1363,Stephen Curry,55
450,Dwyane Wade,53
105,Avery Bradley,52


In [20]:
fouls_on_threes.groupby(['season_x','PLAYER1_NAME_x']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season_x,PLAYER1_NAME_x,count
4182,19,Josh Hart,14
3813,18,Josh Richardson,13
4064,19,De'Aaron Fox,13
4577,20,Kent Bazemore,13
3585,17,Stephen Curry,12
4067,19,DeAndre' Bembry,12
4615,20,Matisse Thybulle,12
4647,20,Patrick Beverley,12
3357,17,Damian Lillard,11
3465,17,Jrue Holiday,11


# The "Master Baiter" Award (sponsored by Bass Pro Shops & Kleenex)

most 3-point shooting fouls drawn (differs from PBPStats)

In [21]:
fouls_on_threes.groupby('PLAYER1_NAME_y').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_y,count
433,James Harden,547
426,Jamal Crawford,243
623,Lou Williams,242
875,Stephen Curry,212
200,Damian Lillard,206
405,JJ Redick,190
566,Kevin Durant,161
554,Kemba Walker,141
596,Kyle Lowry,141
585,Kobe Bryant,139


In [22]:
fouls_on_threes.groupby(['season_y','PLAYER1_NAME_y']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season_y,PLAYER1_NAME_y,count
2149,17,James Harden,121
2592,19,James Harden,95
2816,20,James Harden,70
2360,18,James Harden,69
2190,17,Lou Williams,54
1805,15,Lou Williams,43
1952,16,James Harden,43
1991,16,Lou Williams,43
2928,20,Trae Young,43
2762,20,Damian Lillard,40


In [24]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2016-17",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']].\
assign(season="2016-17")
hustle_seasons=["2017-18","2018-19","2019-20","2020-21","2021-22"]
for seas in hustle_seasons:
    compact_standings=compact_standings.append(leaguestandings.LeagueStandings(league_id='00',
                                             season=seas,season_type="Regular Season").\
                                               standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']].\
                        assign(season=seas))
    print(seas)
    time.sleep(5)

compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

2017-18
2018-19
2019-20
2020-21
2021-22


In [25]:
#from 2015-2016 playoff to current
hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="Totals",season="2016-17",season_type_all_star="Regular Season").hustle_stats_player.get_data_frame().\
assign(season="2016-17")
for seas in hustle_seasons:
    hustle=hustle.append(leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="Totals",season=seas,season_type_all_star="Regular Season").hustle_stats_player.get_data_frame().\
                        assign(season=seas))
    print(seas)
    time.sleep(5)

2017-18
2018-19
2019-20
2020-21
2021-22


In [26]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on=['TEAM_ID','season'],right_on=['TeamID','season'])
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [27]:
hustle_totals=hustle.groupby(['PLAYER_ID','PLAYER_NAME'])[['G','MIN','CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].sum().reset_index()
hustle_totals[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_totals[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_totals.MIN,axis=0).multiply(36,axis=0)

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)

Most charges drawn per 36 minutes (minimum 500 MP and 12 MP/G), credit to morron88 for the idea to separate charges & loose balls in 2020

In [28]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','G','MIN','CHARGES_DRAWN']]

,PLAYER_NAME,G,MIN,CHARGES_DRAWN
119,Marreese Speights,134,1961.0,0.972973
61,Ersan Ilyasova,298,6237.0,0.750361
49,Andrew Bogut,62,933.0,0.694534
728,Moritz Wagner,196,2963.0,0.692541
267,Quincy Acy,118,2040.0,0.564706
283,Aron Baynes,302,5383.0,0.508267
219,Kenneth Faried,130,2485.0,0.507042
279,Willie Reed,113,1459.0,0.493489
550,Ryan Arcidiacono,217,3721.0,0.483741
881,Garrison Mathews,147,2977.0,0.483708


In [29]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['season','G','MIN','PLAYER_NAME','CHARGES_DRAWN']]

,season,G,MIN,PLAYER_NAME,CHARGES_DRAWN
1187,2018-19,67,1231.0,Ersan Ilyasova,1.432981
317,2016-17,82,1286.0,Marreese Speights,0.951788
406,2016-17,73,905.0,Salah Mejri,0.875138
1996,2019-20,58,930.0,Ryan Arcidiacono,0.735484
538,2017-18,64,576.0,Brandon Paul,0.687500
644,2017-18,69,1730.0,Ersan Ilyasova,0.665896
1681,2019-20,68,1677.0,Derrick White,0.644007
1936,2019-20,63,1749.0,Montrezl Harrell,0.617496
146,2016-17,82,2142.0,Ersan Ilyasova,0.605042
1874,2019-20,58,2098.0,Kyle Lowry,0.583413


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 500 MP and 12 MP/G

In [30]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','G','MIN','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,G,MIN,LOOSE_BALLS_RECOVERED
563,Shaquille Harrison,175,2653.0,1.967584
750,Ky Bowman,45,1015.0,1.773399
195,Jeremy Lin,111,2344.0,1.766212
304,Michael Carter-Williams,201,3686.0,1.728703
474,Ben Simmons,275,9326.0,1.710058
451,Treveon Graham,180,2883.0,1.660770
773,Johnathan Williams,39,552.0,1.630435
159,Patrick Beverley,302,8180.0,1.628362
148,Ty Lawson,69,1732.0,1.600462
406,T.J. McConnell,400,9084.0,1.585205


In [31]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['season','G','MIN','PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,season,G,MIN,PLAYER_NAME,LOOSE_BALLS_RECOVERED
931,2017-18,80,2914.0,Russell Westbrook,2.112560
601,2017-18,81,1743.0,Dejounte Murray,2.106713
2572,2020-21,57,551.0,Thanasis Antetokounmpo,2.090744
1433,2018-19,77,2841.0,Paul George,2.027455
1472,2018-19,73,1430.0,Shaquille Harrison,2.013986
564,2017-18,72,1209.0,Corey Brewer,1.995037
1328,2018-19,67,1643.0,Kent Bazemore,1.993914
636,2017-18,63,1809.0,Elfrid Payton,1.950249
526,2017-18,81,2732.0,Ben Simmons,1.950220
366,2016-17,67,2058.0,Patrick Beverley,1.941691


# The Plexiglass Award

most deflections per 36 minutes (minimum 500 MP and 12 MP/G)

In [32]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','G','MIN','DEFLECTIONS']]

,PLAYER_NAME,G,MIN,DEFLECTIONS
863,Matisse Thybulle,196,4270.0,5.201874
516,Gary Payton II,142,2056.0,5.060311
1064,Jose Alvarado,54,834.0,4.705036
284,Nerlens Noel,308,5811.0,4.677336
709,De'Anthony Melton,235,4853.0,4.606635
481,Kris Dunn,245,5897.0,4.358827
309,Robert Covington,393,12173.0,4.258605
987,Facundo Campazzo,130,2609.0,4.194711
43,Tony Allen,93,2187.0,4.164609
406,T.J. McConnell,400,9084.0,4.149273


In [33]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['season','G','MIN','PLAYER_NAME','DEFLECTIONS']]

,season,G,MIN,PLAYER_NAME,DEFLECTIONS
2445,2020-21,65,1298.0,Matisse Thybulle,5.602465
1866,2019-20,51,1269.0,Kris Dunn,5.361702
3033,2021-22,66,1685.0,Matisse Thybulle,5.298516
2825,2021-22,71,1248.0,Gary Payton II,5.192308
1410,2018-19,77,1056.0,Nerlens Noel,5.079545
1945,2019-20,61,1127.0,Nerlens Noel,5.078971
2560,2020-21,69,1796.0,T.J. McConnell,4.910913
392,2016-17,67,2119.0,Robert Covington,4.807928
1664,2019-20,60,1167.0,De'Anthony Melton,4.781491
429,2016-17,81,2133.0,T.J. McConnell,4.708861


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes (minimum 500 MP and 12 MP/G)

In [34]:
hustle_totals.query('MIN >= 500 & MIN/G >= 12').nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','G','MIN','SCREEN_AST_PTS']]

,PLAYER_NAME,G,MIN,SCREEN_AST_PTS
120,Roy Hibbert,48,682.0,16.944282
310,Rudy Gobert,423,13777.0,16.274080
28,Zaza Pachulia,207,3118.0,15.425273
65,Marcin Gortat,211,5380.0,15.417100
293,Cody Zeller,277,6296.0,15.072427
493,Jakob Poeltl,416,8409.0,14.684267
321,Lucas Nogueira,106,1506.0,14.605578
387,Jusuf Nurkic,317,8164.0,13.863792
178,Ed Davis,287,4509.0,13.860279
204,Tristan Thompson,342,8511.0,13.687698


In [35]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['season','G','MIN','PLAYER_NAME','SCREEN_AST_PTS']]

,season,G,MIN,PLAYER_NAME,SCREEN_AST_PTS
79,2016-17,62,1725.0,Cody Zeller,17.697391
2535,2020-21,71,2187.0,Rudy Gobert,17.646091
3124,2021-22,66,2120.0,Rudy Gobert,17.167925
1755,2019-20,72,1326.0,Ivica Zubac,16.805430
1992,2019-20,68,2333.0,Rudy Gobert,16.804115
2237,2020-21,58,966.0,Dwight Powell,16.695652
1024,2017-18,69,972.0,Zaza Pachulia,16.629630
1769,2019-20,66,1171.0,Jakob Poeltl,16.601196
3153,2021-22,76,1999.0,Steven Adams,16.442221
485,2016-17,70,1268.0,Zaza Pachulia,16.296530


In [36]:
player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats(season="1996-97").league_dash_player_bio_stats.get_data_frame()
player_bio["season"]=97
for seas in season_slug:
    new_player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=seas).league_dash_player_bio_stats.get_data_frame()
    new_player_bio["season"]=int(seas[-2:])
    player_bio=player_bio.append(new_player_bio)
    print(seas)
    time.sleep(5)

1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22


# The "David vs Goliath" Award*

most shots blocked with at least a 5-inch difference in height

In [37]:
blocks=pbp[pbp['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','season','PLAYER_HEIGHT_INCHES']],
                                     left_on=['PLAYER1_ID','season'],
                                     right_on=['PLAYER_ID','season']).\
merge(player_bio[['PLAYER_ID','season','PLAYER_HEIGHT_INCHES']],
      left_on=['PLAYER3_ID','season'],
      right_on=['PLAYER_ID','season'])
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
399,Dwyane Wade,261
770,Kyle Lowry,181
422,Eric Bledsoe,132
720,Kemba Walker,128
1097,Russell Westbrook,119
989,Patrick Beverley,117
95,Baron Davis,116
562,James Harden,114
224,Chris Paul,105
654,John Wall,104


In [38]:
blocking_player_heights.query('height_difference >= 5').groupby(['season','PLAYER3_NAME']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,season,PLAYER3_NAME,count
1573,9,Dwyane Wade,36
1936,11,Dwyane Wade,30
4196,22,Patrick Beverley,28
3836,21,Fred VanVleet,26
809,5,Dwyane Wade,25
4088,22,Fred VanVleet,25
1767,10,Dwyane Wade,23
2269,13,Eric Bledsoe,23
2838,16,Kyle Lowry,23
2106,12,Dwyane Wade,21
